# Construcción de la base integrada - Persona 1 (Arquitectura de Datos)

Este notebook crea la base consolidada que integra información ambiental (PM2.5) con variables de salud y contexto demográfico de Bogotá para 2022.  
Será la fuente principal para el modelo multidimensional.


In [3]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 2)

print("Librerías cargadas correctamente.")


Librerías cargadas correctamente.


## 1. Cargar los datos de PM2.5 (I-BOCA)
Estos datos provienen del Observatorio Ambiental de Bogotá.  
Usaremos el archivo limpio generado por el notebook de mi compañero o uno descargado directamente del portal.



In [4]:
# Cargar los datos semestrales de PM2.5
pm1 = pd.read_excel("../data/in/IBOCA-PM25-2022-1.xlsx", skiprows=5)
pm2 = pd.read_excel("../data/in/IBOCA-PM25-2022-2.xlsx", skiprows=5)

# Unir ambos semestres
pm25 = pd.concat([pm1, pm2], ignore_index=True)

# Verificar estructura
pm25.head()


,Fecha & Hora,PM2.5 µg/m3,Unnamed: 2,Unnamed: 3,PM2.5 µg/m3.1,Unnamed: 5,Unnamed: 6,PM2.5 µg/m3.2,Unnamed: 8,Unnamed: 9,PM2.5 µg/m3.3,Unnamed: 11,Unnamed: 12,PM2.5 µg/m3.4,Unnamed: 14,Unnamed: 15,PM2.5 µg/m3.5,Unnamed: 17,Unnamed: 18,PM2.5 µg/m3.6,Unnamed: 20,Unnamed: 21,PM2.5 µg/m3.7,Unnamed: 23,Unnamed: 24,PM2.5 µg/m3.8,Unnamed: 26,Unnamed: 27,PM2.5 µg/m3.9,Unnamed: 29,Unnamed: 30,PM2.5 µg/m3.10,Unnamed: 32,Unnamed: 33,PM2.5 µg/m3.11,Unnamed: 35,Unnamed: 36,PM2.5 µg/m3.12,Unnamed: 38,Unnamed: 39,PM2.5 µg/m3.13,Unnamed: 41,Unnamed: 42,PM2.5 µg/m3.14,Unnamed: 44,Unnamed: 45,PM2.5 µg/m3.15,Unnamed: 47,Unnamed: 48,PM2.5 µg/m3.16,Unnamed: 50,Unnamed: 51,PM2.5 µg/m3.17,Unnamed: 53,Unnamed: 54,PM2.5 µg/m3.18,Unnamed: 56,Unnamed: 57
0,NaN,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA,Concentración,Media móvil,IBOCA
1,2022-01-01 00:00:00,28,13.6,53.31,Sin data,Sin data,Sin data,21,13.3,52.67,49,20.2,67.45,18,5.1,21.25,29,18.5,63.81,17,11.3,47.08,33,15,56.31,46,24.7,77.08,21,11.6,48.33,19,14.9,56.1,28,19.3,65.52,19,13,52.03,41,16.1,58.67,28,14.6,55.45,25,10.8,45,13,11.7,48.75,45,12.3,50.53,Sin data,Sin data,Sin data
2,2022-01-01 01:00:00,30,14.2,54.6,Sin data,Sin data,Sin data,26,14.1,54.38,63,22,71.3,23,5.9,24.58,33,19.3,65.52,32,12,50,27,15.5,57.38,65,26.6,81.15,38,13,52.03,19,15.3,56.95,28,20.3,67.66,22,13.2,52.46,64,18.5,63.81,34,15.6,57.6,28,11.4,47.5,20,12.3,50.53,41,13.5,53.1,Sin data,Sin data,Sin data
3,2022-01-01 02:00:00,40,15.4,57.17,Sin data,Sin data,Sin data,24,14.6,55.45,66,24.2,76.01,18,6.5,27.08,100,22.7,72.8,49,13.3,52.67,52,17.3,61.24,102,29.6,87.58,65,15.3,56.95,26,16.1,58.67,31,21.5,70.23,22,13.6,53.31,79,21.4,70.02,68,17.7,62.09,70,13.5,53.1,19,12.8,51.6,50,15.2,56.74,Sin data,Sin data,Sin data
4,2022-01-01 03:00:00,53,17.2,61.02,Sin data,Sin data,Sin data,48,16.2,58.88,61,26.3,80.51,27,7.4,30.83,74,25.4,78.58,40,14.7,55.67,43,18.6,64.02,35,27.8,83.72,39,16.7,59.95,38,17.5,61.66,35,22.8,73.02,24,14.3,54.81,52,23.1,73.66,50,19.2,65.31,59,15.7,57.81,18,13.4,52.88,61,17.4,61.45,Sin data,Sin data,Sin data


## 2. Asignar localidades a cada estación
El dataset de IBOCA usa nombres de estaciones.  
Debemos crear un catálogo que relacione cada estación con la localidad donde está ubicada.


In [31]:
import pandas as pd
import numpy as np
import unicodedata
import re
from pathlib import Path

# ---------- 1) utilidades ----------
def norm(x: str) -> str:
    """Normaliza texto: mayúsculas, sin acentos, sin dobles espacios."""
    if pd.isna(x): 
        return x
    x = unicodedata.normalize("NFKD", str(x)).encode("ascii","ignore").decode("ascii")
    return " ".join(x.upper().strip().split())

def read_iboca_pm25_long(xlsx_path: str) -> pd.DataFrame:
    """
    Lee un Excel de IBOCA (PM2.5) con cabeceras/metadata variables y devuelve
    un DataFrame largo: DateTime, Station, Value
    """
    df0 = pd.read_excel(xlsx_path, header=None)

    # localiza fila donde aparece "Estac" (fila que anuncia el bloque de estaciones)
    row_est = df0.index[
        df0.apply(lambda r: r.astype(str).str.contains("Estac", case=False, na=False).any(), axis=1)
    ][0]

    # en estos reportes, justo dos filas debajo están los nombres de las estaciones
    hdr_row = row_est + 2

    header = df0.iloc[hdr_row].astype(str).str.strip().tolist()
    header[0] = "DateTime"
    df_raw = df0.iloc[hdr_row+1:].copy()
    df_raw.columns = header
    df_raw = df_raw.dropna(axis=1, how="all")
    df_raw = df_raw[df_raw["DateTime"].notna()]

    # tipos correctos
    df_raw["DateTime"] = pd.to_datetime(df_raw["DateTime"], errors="coerce")
    df_raw = df_raw[df_raw["DateTime"].notna()]

    station_cols = [c for c in df_raw.columns if c != "DateTime"]

    pm25_long = df_raw.melt(
        id_vars="DateTime",
        value_vars=station_cols,
        var_name="Station",
        value_name="Value"
    )

    # limpieza de valores
    pm25_long["Station"] = pm25_long["Station"].astype(str).str.strip()
    pm25_long["Value"] = (
        pm25_long["Value"].astype(str)
        .str.replace(",", ".", regex=False)
        .str.replace(r"[^\d\.]", "", regex=True)
        .replace("", np.nan)
        .astype(float)
    )
    pm25_long = pm25_long.dropna(subset=["Value"])
    return pm25_long

def auto_localidad_from_station(st_name: str) -> str | None:
    """
    Reglas de mapeo de estación -> Localidad usando palabras clave (regex).
    No requiere archivos externos.
    Devuelve nombre de localidad (Title Case) o None si no coincide.
    """
    s = norm(st_name)

    # excluir estaciones móviles explícitamente
    if re.search(r"\bMOVIL\b", s):
        return None

    # ---- reglas por localidad (puedes ampliar si te aparece alguna nueva) ----
    rules = {
        "Usaquén":       [r"\bUSAQUEN\b", r"\bTOBERIN\b", r"\bUSAQUEN-?TOBERIN\b"],
        "Chapinero":     [r"\bCHAPINERO\b"],
        "Santa Fe":      [r"\bSANTA\s*FE\b"],
        "San Cristóbal": [r"\bSAN\s*CRISTOBAL\b"],
        "Usme":          [r"\bUSME\b"],
        "Tunjuelito":    [r"\bTUNAL\b", r"\bTUNJUELITO\b"],
        "Bosa":          [r"\bBOSA\b"],
        "Kennedy":       [r"\bKENNEDY\b", r"\bCARVAJAL\b", r"\bSEVILLANA\b", r"CARVAJAL\s*-\s*SEVILLANA"],
        "Fontibón":      [r"\bFONTIBON\b"],
        "Engativá":      [r"\bENGATIVA\b", r"\bBOLIVIA\b", r"\bLAS\s*FERIAS\b"],
        "Suba":          [r"\bSUBA\b", r"\bCOLINA\b", r"\bGUAYMARAL\b", r"SUBA\s*-\s*COMPARTIR"],
        "Barrios Unidos":[r"\bBARRIOS\s*UNIDOS\b", r"\bCDAR\b"],
        "Teusaquillo":   [r"\bTEUSAQUILLO\b", r"\bMINAMBIENTE\b", r"\bMINISTERIO\b"],
        "Los Mártires":  [r"\bMARTIRES\b"],
        "Antonio Nariño":[r"\bANTONIO\s*NARINO\b"],
        "Puente Aranda": [r"\bPUENTE\s*ARANDA\b", r"\bJAZMIN\b"],
        "La Candelaria": [r"\bCANDELARIA\b"],
        "Rafael Uribe Uribe":[r"\bRAFAEL\s*URIBE\b"],
        "Ciudad Bolívar":[r"\bCIUDAD\s*BOLIVAR\b"],
        "Sumapaz":       [r"\bSUMAPAZ\b"],
    }

    for loc, patterns in rules.items():
        for p in patterns:
            if re.search(p, s):
                return loc

    # sin match
    return None

# ---------- 2) lee S1 + S2 y concatena ----------
in_dir = Path("../data/in")
pm25_s1 = read_iboca_pm25_long(in_dir/"IBOCA-PM25-2022-1.xlsx")
pm25_s2 = read_iboca_pm25_long(in_dir/"IBOCA-PM25-2022-2.xlsx")
pm25_long_all = pd.concat([pm25_s1, pm25_s2], ignore_index=True).drop_duplicates(["DateTime","Station"])

# ---------- 3) mapeo 100% automático de Localidad ----------
pm25_long_all["Localidad"] = pm25_long_all["Station"].apply(auto_localidad_from_station)

# reporta cobertura del mapeo
total_est = pm25_long_all["Station"].nunique()
mapeadas = pm25_long_all.loc[pm25_long_all["Localidad"].notna(), "Station"].nunique()
print(f"Estaciones distintas: {total_est} | mapeadas: {mapeadas} | sin mapa: {total_est - mapeadas}")

# si quieres ver cuáles no se mapean (para enriquecer reglas)
unmapped = (pm25_long_all.loc[pm25_long_all["Localidad"].isna(), "Station"]
            .drop_duplicates().sort_values())
print("Ejemplos sin mapa:", unmapped.head(15).to_list())

# ---------- 4) excluir filas sin localidad (móviles o desconocidas) ----------
pm25 = pm25_long_all.dropna(subset=["Localidad"]).copy()

# ---------- 5) agrega trimestre y calcula promedio ----------
pm25["Trimestre"] = pm25["DateTime"].dt.quarter
pm25_localidad = (pm25
    .groupby(["Localidad","Trimestre"])["Value"]
    .mean()
    .reset_index()
    .rename(columns={"Value":"Promedio_PM25"})
)

print("Filas agregadas:", len(pm25_localidad))
pm25_localidad.head(10)

# ---------- 6) guarda salidas ----------
out_dir = Path("../data/out")
out_dir.mkdir(parents=True, exist_ok=True)
pm25_long_all.to_csv(out_dir/"PM25_long_2022.csv", index=False)
pm25_localidad.to_csv(out_dir/"PM25_por_localidad_trimestre.csv", index=False)
print("✅ Guardados:\n -", out_dir/"PM25_long_2022.csv", "\n -", out_dir/"PM25_por_localidad_trimestre.csv")


C:\Users\camil\AppData\Local\Temp\ipykernel_13152\1228967737.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw["DateTime"] = pd.to_datetime(df_raw["DateTime"], errors="coerce")
C:\Users\camil\AppData\Local\Temp\ipykernel_13152\1228967737.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw["DateTime"] = pd.to_datetime(df_raw["DateTime"], errors="coerce")


Estaciones distintas: 20 | mapeadas: 17 | sin mapa: 3
Ejemplos sin mapa: ['Móvil 7ma', 'Móvil Fontibón', 'nan']
Filas agregadas: 48
✅ Guardados:
 - ..\data\out\PM25_long_2022.csv 
 - ..\data\out\PM25_por_localidad_trimestre.csv


In [39]:
import pandas as pd
import numpy as np
import unicodedata

pd.set_option("display.max_rows", 100)

def norm_txt(x: str) -> str:
    if pd.isna(x): 
        return x
    x = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode("ascii")
    return " ".join(x.strip().title().split())

# Lista de localidades válidas (para filtrar “Distrito”, “Sin Información”, etc.)
LOCALIDADES_BOG = {
    "Usaquen","Chapinero","Santa Fe","San Cristobal","Usme","Tunjuelito","Bosa",
    "Kennedy","Fontibon","Engativa","Suba","Barrios Unidos","Teusaquillo",
    "Los Martires","Antonio Nariño","Puente Aranda","La Candelaria",
    "Rafael Uribe Uribe","Ciudad Bolivar","Sumapaz"
}


In [41]:
pm = pd.read_csv("../data/out/PM25_por_localidad_trimestre.csv")
pm["Localidad"] = pm["Localidad"].apply(norm_txt)
pm_2022 = pm.copy()  # ya es 2022
pm_2022.head(3)


,Localidad,Trimestre,Promedio_PM25
0,Barrios Unidos,1,18.36
1,Barrios Unidos,2,13.28
2,Barrios Unidos,3,12.90


In [54]:
import pandas as pd
import numpy as np
import unicodedata

# ---------- lector robusto para tus CSV (quita BOM y tildes en headers) ----------
def read_csv_robusto(path, sep=";"):
    for enc in ("utf-8-sig", "latin-1", "utf-8"):
        try:
            df = pd.read_csv(path, sep=sep, encoding=enc)
            break
        except UnicodeDecodeError:
            continue
    def norm_col(c):
        c = str(c).replace("\ufeff", "").strip()
        c = unicodedata.normalize("NFKD", c).encode("ascii","ignore").decode("ascii")
        return c.upper()
    df.columns = [norm_col(c) for c in df.columns]
    return df

def norm_loc(s):
    if pd.isna(s): return s
    s = str(s).strip()
    s = s.replace("\ufeff","")
    s = s.replace("�","")
    s = s.replace("Ã±","ñ")
    s = s.replace("Ã¡","á").replace("Ã©","é").replace("Ã­","í").replace("Ã³","ó").replace("Ãº","ú")
    # quita "00 - " si viene
    s = pd.Series(s).str.replace(r"^\s*\d+\s*-\s*", "", regex=True).iloc[0]
    return s.title()

LOCALIDADES_BOG = {
    "Usaquen","Chapinero","Santa Fe","San Cristobal","Usme","Tunjuelito","Bosa",
    "Kennedy","Fontibon","Engativa","Suba","Barrios Unidos","Teusaquillo",
    "Los Martires","Antonio Nariño","Puente Aranda","La Candelaria",
    "Rafael Uribe Uribe","Ciudad Bolivar","Sumapaz"
}

# ---------- PM2.5 ya lo tienes en pm_2022 ----------

# ---------- 1) Bajo peso ----------
bp = read_csv_robusto("../data/in/bajo_peso.csv")
bp["ANO"] = pd.to_numeric(bp["ANO"], errors="coerce")
bp = bp[bp["ANO"] == 2022].copy()
bp["LOCALIDAD"] = bp["LOCALIDAD"].apply(norm_loc)
bp = bp[bp["LOCALIDAD"].isin(LOCALIDADES_BOG)]
for col in ["TOTAL_NACIDOS_VIVOS","TOTAL_BPN"]:
    bp[col] = pd.to_numeric(bp[col], errors="coerce")
bp_loc = (bp.groupby("LOCALIDAD")[["TOTAL_NACIDOS_VIVOS","TOTAL_BPN"]]
            .sum().reset_index()
            .rename(columns={"LOCALIDAD":"Localidad","TOTAL_NACIDOS_VIVOS":"Nacidos","TOTAL_BPN":"Bajo_Peso"}))
bp_loc["Porcentaje_Bajo_Peso"] = (bp_loc["Bajo_Peso"]/bp_loc["Nacidos"]*100).round(3)

# ---------- 2) Natalidad (para sexo y edad materna) ----------
nat = read_csv_robusto("../data/in/osb_tnatalidad.csv")
nat["ANO"] = pd.to_numeric(nat["ANO"], errors="coerce")
nat = nat[nat["ANO"] == 2022].copy()
nat["Localidad"] = nat["LOCALIDAD_MADRE"].apply(norm_loc)
nat = nat[nat["Localidad"].isin(LOCALIDADES_BOG)]
nat["TOTAL_NACIDOS"] = pd.to_numeric(nat["TOTAL_NACIDOS"], errors="coerce").fillna(0)

# 2a) Proporción de masculino
sexo = (nat[nat["SEXO"].str.lower().isin(["masculino","femenino"])]
        .groupby(["Localidad","SEXO"])["TOTAL_NACIDOS"].sum()
        .unstack(fill_value=0).reset_index())
if "Masculino" not in sexo.columns: sexo["Masculino"] = 0
if "Femenino"  not in sexo.columns: sexo["Femenino"]  = 0
den = (sexo["Masculino"] + sexo["Femenino"]).replace(0, np.nan)
sexo["Prop_Masculino"] = (sexo["Masculino"]/den).round(4)
sexo = sexo[["Localidad","Prop_Masculino"]]

# 2b) Edad materna promedio ponderada por nacidos
nat["EDAD_MADRE"] = pd.to_numeric(nat["EDAD_MADRE"], errors="coerce")
edad_mat = (nat.groupby("Localidad")
            .apply(lambda g: np.average(g["EDAD_MADRE"], weights=g["TOTAL_NACIDOS"])
                             if g["TOTAL_NACIDOS"].sum()>0 else np.nan)
            .reset_index(name="Edad_Materna_Prom"))
edad_mat["Edad_Materna_Prom"] = edad_mat["Edad_Materna_Prom"].round(2)

# ---------- 3) Mortalidad infantil ----------
mi = read_csv_robusto("../data/in/osb_tm_infantil.csv")
mi["ANIO"] = pd.to_numeric(mi["ANIO"], errors="coerce")
mi = mi[mi["ANIO"] == 2022].copy()
mi["Localidad"] = mi["LOCALIDAD"].apply(norm_loc)
mi = mi[mi["Localidad"].isin(LOCALIDADES_BOG)]
tasa_col = next(c for c in mi.columns if "TASA" in c)  # p.ej. "TASA POR 1000"
mi[tasa_col] = pd.to_numeric(mi[tasa_col], errors="coerce")
mi_loc = (mi.groupby("Localidad")[tasa_col].mean()
          .reset_index().rename(columns={tasa_col:"Mort_Infantil"}))


C:\Users\camil\AppData\Local\Temp\ipykernel_13152\105602624.py:74: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.average(g["EDAD_MADRE"], weights=g["TOTAL_NACIDOS"])


In [55]:
# Partimos de PM etiquetado (ya cargado en pm_2022)
base = pm_2022.rename(columns={"Promedio_PM25":"PM25_prom_trimestre"}).copy()

# Une solo los que existan
dfs_disponibles = []
for name in ["bp_loc","sexo","edad_mat","mi_loc"]:
    if name in locals():
        dfs_disponibles.append(locals()[name])
    else:
        print(f"⚠️  Ojo: {name} no está definido, se omite en el merge.")

for df in dfs_disponibles:
    base = base.merge(df, on="Localidad", how="left")

# Orden sugerido
cols = ["Localidad","Trimestre","PM25_prom_trimestre","PM25_prom_anual",
        "Categoria_Exposicion","Nacidos","Bajo_Peso","Porcentaje_Bajo_Peso",
        "Prop_Masculino","Edad_Materna_Prom","Mort_Infantil"]
base = base[[c for c in cols if c in base.columns]].copy()

base.head(10)


,Localidad,Trimestre,PM25_prom_trimestre,Nacidos,Bajo_Peso,Porcentaje_Bajo_Peso,Prop_Masculino,Edad_Materna_Prom,Mort_Infantil
0,Barrios Unidos,1,18.36,1008.0,143.0,14.19,0.53,29.31,10.9
1,Barrios Unidos,2,13.28,1008.0,143.0,14.19,0.53,29.31,10.9
2,Barrios Unidos,3,12.90,1008.0,143.0,14.19,0.53,29.31,10.9
3,Barrios Unidos,4,17.14,1008.0,143.0,14.19,0.53,29.31,10.9
4,Ciudad Bolivar,1,24.25,NaN,NaN,NaN,NaN,NaN,NaN
5,Ciudad Bolivar,2,16.38,NaN,NaN,NaN,NaN,NaN,NaN
6,Ciudad Bolivar,3,14.91,NaN,NaN,NaN,NaN,NaN,NaN
7,Ciudad Bolivar,4,20.48,NaN,NaN,NaN,NaN,NaN,NaN
8,Engativa,1,19.60,NaN,NaN,NaN,NaN,NaN,NaN
9,Engativa,2,13.71,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
base.to_csv("../data/out/base_integrada_2022.csv", index=False)
print("✅ Base integrada lista en ../data/out/base_integrada_2022.csv")


✅ Base integrada lista en ../data/out/base_integrada_2022.csv
